In [1]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer
from rasterio.warp import reproject, Resampling, CRS
import pyproj

In [2]:
#LOAD RASTER DATA
#from rasterio import features
with rasterio.open('src_raster/hijo_green.tif') as dataset:
    rband = dataset.read(1).astype(np.float32)
    gband = dataset.read(2).astype(np.float32)
    bband = dataset.read(3).astype(np.float32)
    meta_vari = dataset.meta
    meta_exg = dataset.meta
    meta_gli = dataset.meta

In [3]:
meta_vari.update(nodata=0.0, dtype=rasterio.float32)
meta_exg.update(nodata=0.0, dtype=rasterio.float32)
meta_gli.update(nodata=0.0, dtype=rasterio.float32)

In [ ]:
meta_vari

In [4]:
#VARI
denominator = gband + rband - bband
denominator = np.where(np.isclose(denominator, 0), 1e-1, denominator)  # Avoid division by nearly zero
vari = (gband - rband) / denominator
# Replace NaN values with nodata
vari[np.isnan(vari)] = meta_vari['nodata']
vari[vari==0] = meta_vari['nodata']
vari = np.clip(vari, -1, 1)

In [5]:
#GLI
gli_denominator = 2 * gband + rband + bband
gli_denominator = np.where(np.isclose(gli_denominator, 0), 1e-1, gli_denominator)
gli = (2* gband - rband - bband)/ gli_denominator

# Replace NaN values with nodata
gli[np.isnan(gli)] = meta_gli['nodata']
gli[gli== 0] = meta_gli['nodata']

gli = np.clip(gli, -1, 1)

In [6]:
#ExG
exg = 2 * gband - rband - bband
exg[np.isnan(exg)] = meta_exg['nodata']
exg[exg==0] = meta_exg['nodata']

In [8]:
#VARI EXPORT
new_tiff_profile_vari = dataset.profile  # Copy the profile from the original dataset
new_tiff_profile_vari.update(
                        dtype=rasterio.float32,  # Update the data type to match the VARI data
                        count=1,  # Only one band for VARI
                        compress='lzw',  # You can choose a compression method if needed
                        tiled=False,
                        blockysize=1,
                        nodata=0.0
)
new_crs = pyproj.CRS.from_epsg(32651)
new_tiff_profile_vari.update(crs=new_crs)

#export data to file
with rasterio.open('out_raster/index/hijo_vari_green.tif', 'w', **new_tiff_profile_vari) as new_tiff:
                            new_tiff.write(vari, 1)

In [9]:
#GLI EXPORT
new_tiff_profile_gli = dataset.profile  # Copy the profile from the original dataset
new_tiff_profile_gli.update(
                        dtype=rasterio.float32,  # Update the data type to match the VARI data
                        count=1,  # Only one band for VARI
                        compress='lzw',  # You can choose a compression method if needed
                        tiled=False,
                        blockysize=1,
                        nodata=0.0
)
new_crs = pyproj.CRS.from_epsg(32651)
new_tiff_profile_gli.update(crs=new_crs)

#export data to file
with rasterio.open('out_raster/index/hijo_gli_green.tif', 'w', **new_tiff_profile_gli) as new_tiff:
                            new_tiff.write(gli, 1)

In [10]:
#EXG EXPORT
new_tiff_profile_exg = dataset.profile  # Copy the profile from the original dataset
new_tiff_profile_exg.update(
                        dtype=rasterio.float32,  # Update the data type to match the VARI data
                        count=1,  # Only one band for VARI
                        compress='lzw',  # You can choose a compression method if needed
                        tiled=False,
                        blockysize=1,
                        nodata=0.0
)
new_crs = pyproj.CRS.from_epsg(32651)
new_tiff_profile_exg.update(crs=new_crs)

#export data to file
with rasterio.open('out_raster/index/hijo_exg_green.tif', 'w', **new_tiff_profile_exg) as new_tiff:
                            new_tiff.write(exg, 1)